This lab covers authentication and access control. 

Topics:
- Password strength analysis
- Password hashing methods
- Salt and pepper implementation
- TOTP (Time-based One Time Password) (2FA) implementation
- Brute force attack simulation
- Complete auth system

In [23]:
%pip install bcrypt pyotp qrcode pillow

# bcrypt: For hashing passwords securely
# pyotp: For generating and verifying time-based one-time passwords (TOTP)
# qrcode: For generating QR codes
# pillow: For image processing, used by qrcode to create QR code images


Note: you may need to restart the kernel to use updated packages.


# Section 1: Password Strength
Below is a function that acts as a "password meter". It uses a scoring system based on common best practices.

- Length check: awards points for meeting 8+ and 12+ character minimums.
- Character variety: it checks for the presence of uppercase, lowercase, numbers, and special symbols (string.punctuation), awarding a point for each.
- Common password check: it checks against a short list of notoriously bad passwords (such as "Have I Been Pwned" list)
- Entropy: a measure of randomness or unpredicability. In passwords, higher entropy means it's harder to guess. You can use this approximation to calculate it: 

```Approximate entropy: length * log2(pool_size)```

Where pool_size is the number of options that a password can be for example if only the lower-case alphabets, then the pool size would be 26.

In [4]:
import string, math

while True:
    password = input("Enter your password: ")
    print("Password: ", password)

    strength = 0
    pool_size = 0

    # adds points based on length, character/symbol/number variety
    if len(password) >= 8:
        strength += 1
        if len(password) >= 12:
            strength += 1
            if len(password) >= 16:
                strength += 1
    if any(char.isupper() for char in password):
        strength += 1
        pool_size += 26
    if any(char.islower() for char in password):
        strength += 1
        pool_size += 26
    if any(char.isdigit() for char in password):
        strength += 1
        pool_size += 10
    if any(char in string.punctuation for char in password):
        strength += 1
        pool_size += 32

    common_passwords = {"password", "123456", "123456789", "qwerty", "abc123"}

    # checks if password is in common passwords list
    if password.lower() in common_passwords:
        strength = 0

    # calculates approximate entropy
    approximate_entropy = len(password) * math.log2(pool_size)

    print("Password entropy (bits):", round(approximate_entropy, 2))

    if approximate_entropy >= 25:
        strength += 1
        if approximate_entropy >= 50:
            strength += 1
            if approximate_entropy >= 75:
                strength += 1
                if approximate_entropy >= 100:
                    strength += 1
                    if approximate_entropy >= 125:
                        strength += 1
                        if approximate_entropy >= 150:
                            strength += 1

    print("Password strength:", strength)

Password:  cheese
Password entropy (bits): 28.2
Password strength: 2
Password:  CheesE
Password entropy (bits): 34.2
Password strength: 3
Password:  Cheese2004
Password entropy (bits): 59.54
Password strength: 6
Password:  TheBigCheese%&@2005
Password entropy (bits): 124.54
Password strength: 11
Password:  


ValueError: math domain error

# Section 2: Hashes
This section compares insecure hashing algorithms such as MD5, SHA-256 against a secure algorithm like bcrypt.

Hashing is a one way cryptographic function that transforms an input such as a password into a **fixed-size** string of characters called a hash. It's designed to be irreversible.

MD5, & SHA-256 are fast hashing algorithms. Their speed is great for checking the file integrity, but terrible for passwords, as it allows attackers to guess billions of passwords per second (brute force attack).

bcrypt is a password hashing function that is intentionally slow and adaptive. You can increase the "work factor" (or rounds) over time as computers get faster, keeping your hashes secure.

In [14]:
import sys
import hashlib, bcrypt

# hashes password using MD5
def hash_password_md5(password: str) -> str:
    md5_hash = hashlib.md5(password.encode())
    return md5_hash.hexdigest() # hexdigest returns the hash in hexadecimal format

# hashes password using SHA-256
def hash_password_sha256(password: str) -> str:
    sha256_hash = hashlib.sha256()
    sha256_hash.update(password.encode())
    return sha256_hash.hexdigest()

# hashes password using bcrypt
def hash_password_bcrypt(password: str) -> str:
    #converting password to bytes
    bytes = password.encode()
    # generating salt
    salt = bcrypt.gensalt()
    # hashing password
    bcrypt_hash = bcrypt.hashpw(bytes, salt)
    return bcrypt_hash

stored_password = input("Enter password: ")

MD5_password = hash_password_md5(stored_password)
print("MD5 Hashed password: ", MD5_password)
SHA256_password = hash_password_sha256(stored_password)
print("SHA-256 Hashed password: ", SHA256_password)
Bcrypt_password = hash_password_bcrypt(stored_password)
print("Bcrypt Hashed password: ", Bcrypt_password)

entered_password = input("Re-enter password: ")

# comparing entered password with stored hashed passwords
if hash_password_md5(entered_password) == MD5_password:
    print("MD5: Password match!")
else:
    print("MD5: Password do not match!")

if hash_password_sha256(entered_password) == SHA256_password:
    print("SHA-256: Password match!")
else:
    print("SHA-256: Password do not match!")    

# bcrypt has a built-in method to check passwords. It required the entered password in bytes.
if bcrypt.checkpw(entered_password.encode(), Bcrypt_password):
    print("Bcrypt: Password match!")
else:
    print("Bcrypt: Password do not match!")


MD5 Hashed password:  fea0f1f6fede90bd0a925b4194deac11
SHA-256 Hashed password:  873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
Bcrypt Hashed password:  b'$2b$12$ajMrVuitPJAOeM2XhriqvORAgV8lM0/JTjGZdLX6ysOjfFhUJDQmW'
MD5: Password match!
SHA-256: Password match!
Bcrypt: Password match!


Why is a fast algorithm bad for storing passwords? 

It makes brute-force attacks much faster for an attacker who steals the hash database. Fast algorithms like MD5 and SHA-256 were designed for speed, which is great for checksums, but terrible for passwords. Modern GPUs can try billions of MD5/SHA-256 hashes *per second.*

- Weak passwords are cracked almost instantly
- Long and complex passwords become vulnerable with enough hardware.
- Attackers can cheaply scale brute force attacks.

Slow algorithms such as **bcrypt** are intentionally designed to be slow and computationally expensive and waste time. They include configurable work fasctors and memory hardness to limit how many guesses an attacker can make per second, even with specialised hardware. This drastically increases the time and cost required to crack passwords.

# Section 3: Salt and Pepper
Section 3 is on salt and pepper. Salting defeats pre-computation attacks (such as rainbow tables). Pepper adds another layer of security.

- Rainbow table: a precomputed table of hashes for common passwords. If you don't use a salt, an attacker can find a matching hash in their table and instantly know your password.
- Salt: A unique, random string that is added to a password before hashing. Every user gets a different salt. It is stored in the database along side the hashed password, so that it can be added to the user's input whenever they log in.
- Pepper: a secret, static string that is added to a password before hashing. Unlike a salt, the pepper is the same for all users and is NOT stored in the databse. It's stored securely in the application's configuration. or an environment variable.

The code below hashes a password first without a salt, then with a salt, then with pepper.

In [22]:
import os
import hashlib

# This demonstrates the differences between hashing with and without salts when the same password is used.
password1 = "SuperSecurePassword123"
password2 = "SuperSecurePassword123"

# hashes password using SHA-256 no salt or pepper.
def hash_sha256(password: str) -> str:
    sha256_hash = hashlib.sha256()
    sha256_hash.update(password.encode())
    return sha256_hash.hexdigest()

# hashes password using SHA-256 with a random salt.
def hash_sha256_salt(password: str) -> str:
    # Generate a secure random salt (16 bytes).
    # Using os.urandom returns n random bytes using the OS's cryptographic random number generator.
    # It's not a good idea to use python's random module, as its output is deterministic, and can be predicted.
    salt = os.urandom(16)

    sha256_hash = hashlib.sha256()
    sha256_hash.update(salt + password.encode())
    return sha256_hash.hexdigest(), salt.hex()

def hash_sha256_salt_pepper(password: str, pepper: str) -> str:
    # Generate a secure random salt (16 bytes).
    salt = os.urandom(16)

    # combine salt + password + pepper
    sha256_hash = hashlib.sha256()
    sha256_hash.update(salt + password.encode() + pepper.encode())
    return sha256_hash.hexdigest(), salt.hex()


# hashing passwords without salt

hashed_no_salt_1 = hash_sha256(password1)
print("Hashing password with no salt:", hashed_no_salt_1)

hashed_no_salt_2 = hash_sha256(password2)
print("Re-hashing same password with no salt:", hashed_no_salt_2)

# hashes should match when no salt is used
if hashed_no_salt_1 == hashed_no_salt_2:
    print("Hashes match. Both passwords give the same hash.")
else:
    print("Hashes do not match.")


# hashing passwords with salt

hashed_salt_1 = hash_sha256_salt(password1)
print("\nHashing password with salt.")
print("Hash:", hashed_salt_1[0])
print("Salt:", hashed_salt_1[1])

hashed_salt_2 = hash_sha256_salt(password2)
print("\nRe-hashing same password with new salt.")
print("Hash:", hashed_salt_2[0])
print("Salt:", hashed_salt_2[1])

# hashes should not match when different salts are used
if hashed_salt_1[0] == hashed_salt_2[0]:
    print("\nHashes match. Both passwords give the same hash.")
else:
    print("\nHashes do not match. Salts ensure different hashes even for the same password.")

# hashing passwords with salt and pepper

# a pepper value should be stored in a secure config file or env.
# for the sake of this demo, the pepper is defined here.
PEPPER = "secure_pepper_value"

hashed_salt_pepper_1 = hash_sha256_salt_pepper(password1, PEPPER)
print("\nHashing password with salt and pepper.")
print("Hash:", hashed_salt_pepper_1[0])
print("Salt:", hashed_salt_pepper_1[1])
print("Pepper:", PEPPER)

hashed_salt_pepper_2 = hash_sha256_salt_pepper(password2, PEPPER)
print("\nRe-hashing same password with new salt and same pepper.")
print("Hash:", hashed_salt_pepper_2[0])
print("Salt:", hashed_salt_pepper_2[1])
print("Pepper:", PEPPER)

# hashes should not match when different salts are used, even with the same pepper
if hashed_salt_pepper_1[0] == hashed_salt_pepper_2[0]:
    print("\nHashes match. Both passwords give the same hash.")
else:
    print("\nHashes do not match. Salts ensure different hashes even for the same password, even with the same pepper.")    

Hashing password with no salt: 00a623b8ec76d8149b45589005bd1b9430b43a24c4ffb325e15ec410c738ec5a
Re-hashing same password with no salt: 00a623b8ec76d8149b45589005bd1b9430b43a24c4ffb325e15ec410c738ec5a
Hashes match. Both passwords give the same hash.

Hashing password with salt.
Hash: 9c6da477097ddefc6319f40303e85bb1bcb5d16d5084e6cbfff061ba07e0c9b1
Salt: 93df3a1e89201bf4eaeddcce5a9882d8

Re-hashing same password with new salt.
Hash: ca17c1d28e036e072e1537b70e3f2779b20230a88a777a61e0745b4cb30c57e2
Salt: 5e6b55808d9ce48eaccffec61e365370

Hashes do not match. Salts ensure different hashes even for the same password.

Hashing password with salt and pepper.
Hash: b10583a43c062dcfdbbf591fc67c74a283e252f501f7efad0ffed68a82b16875
Salt: a022c1eb4031060301b9f00e0d0f68ae
Pepper: secure_pepper_value

Re-hashing same password with new salt and same pepper.
Hash: 3274e17dcd0ef765f7de1907741f108f76a05d7c1c7ed21aedbf380dc52381e6
Salt: 927ebba406372b9a59fb9e09c49fad70
Pepper: secure_pepper_value

Hashes 

| Method                     | Rainbow Table Attacks | Brute Force Difficulty            | Requires Server Compromise? | Overall Safety |
|----------------------------|-----------------------|-----------------------------------|-----------------------------|----------------|
| **SHA-256, No Salt**       | Easy                  | Very easy (billions/sec)          | No                          | Bad            |
| **SHA-256 + Salt**         | Impossible            | Medium (still fast hashing)       | No                          | OK             |
| **SHA-256 + Salt + Pepper**| Impossible            | Very hard unless pepper stolen    | Yes                         | Good           |
| **bcrypt**                 | Impossible            | Slow by design (best)             | No                          | Excellent      |


# Section 4: Two-Factor Authentication (2FA) - Time-based One-Time Passwords (TOTP)

Two-Factor Authentication can be added as a second layer of security using Time-based One-Time Passwords.

- 2FA (Two-Factor Authentication): A security process where users provide two different authentication factors. This is typically "something you know" (password) and "something you have" (your
phone).
- TOTP (Time-based One-Time Password): An algorithm that uses a shared secret key and the current time to generate a temporary, single-use code.
- Provisioning URI: A special link (often shown as a QR code) that contains the secret key, account name, and issuer information needed for an authenticator app (like Google Authenticator or Authy) to start generating codes.

The cell below demonstrates how OTPs work.

In [ ]:
import pyotp
import time

# Create a TOTP object with a base32 secret.
totp = pyotp.TOTP('base32secret3232')
totp.now()

one_time_password = totp.now()
print("OTP:", one_time_password)

# OTP verified for current time
if (totp.verify(one_time_password) == True):
    print("OTP is valid.")
else:
    print("OTP is invalid.")

time.sleep(30) # wait for the OTP to expire

# OTP verified after 30 seconds
if (totp.verify(one_time_password) == True):
    print("OTP is valid.")
else:
    print("OTP is invalid.")

OTP: 284806
OTP is valid.
OTP is invalid.


The cell below does the same thing, instead letting your test the OTP yourself. You can enter the code given to verify. If you take longer than 30 seconds to enter the code, it will expire and fail to verify.

In [ ]:
import pyotp
import time

# Create a TOTP object with a base32 secret.
totp = pyotp.TOTP('base32secret3232')
totp.now()

print(totp.now())

# Prompt user to enter the OTP
one_time_password = input("Enter the OTP: ")

# OTP verified for current time
if (totp.verify(one_time_password) == True):
    print("OTP is valid.")
else:
    print("OTP is invalid.")

935628
OTP is invalid.


In [ ]:
# You can generate a random secret key using this helper function.
secret_key_32 = pyotp.random_base32()

secret_key_hex = pyotp.random_hex()

print("Random Base32 Secret Key:", secret_key_32)
print("Random Hex Secret Key:", secret_key_hex)

Random Base32 Secret Key: 5A6FRZFS6V5WP2EWFVBJAFK6GCMHWPXG
Random Hex Secret Key: CFBE7C82738841D96BBAD04131476EC355C82775


### Google Authenticator compatible OTP
PyOTP works with the Google Authenticator app, as well as other OPT apps like Authy. PyOTP includes the ability to generate provisioning URIs for use with the QR code scanner built into the these MFA client apps.

```pyotp.totp.TOTP('JBSWY3DPEHPK3PXP')```
This line creates a TOTP generator. The string is the shared Base32-encoded secret key, used by both the server and authenticator apps to generate matching codes.

```.provisioning_uri(name='alice@google.com', issuer_name='Secure App')```
This line generates a **provisioning URI** based on the TOTP configuration. This URI is used to create QR codes that apps like Google Authenticator can scan to configure the 2FA entry automatically.
- ```name='alice@google.com'``` is the user account associated with the OTP.
- ```issuer_name='Secure App'``` is the name of the service generating the OTP which will be shown in the authenticator app.

In [8]:
pyotp.totp.TOTP('JBSWY3DPEHPK3PXP').provisioning_uri(name='alice@google.com', issuer_name='Secure App')

pyotp.parse_uri('otpauth://totp/Secure%20App:alice%40google.com?secret=JBSWY3DPEHPK3PXP&issuer=Secure%20App')


### Section 5: Simulating a brute-force attack

This section visually demonstrates why fast hashes are weak and why slow, salted hashes are strong.

- Brute-force attack: an attack where a threat actor tries every possible combination until they find the correct one.

- Dictionary attack: a more efficient form of brute-force where the attacker uses a list of common words and passwords (a "dictionary" of "wordlist")

The code below asks the user to input a password and hashes it using SHA-256.

In [ ]:
import os, hashlib, time

def hash_sha256(password: str) -> str:
    sha256_hash = hashlib.sha256()
    sha256_hash.update(password.encode())
    return sha256_hash.hexdigest()

secret_password = input("Enter a password to crack: ")

hashed_password = hash_sha256(secret_password)

file = open("100k_common_passwords.txt", "r")

counter = 0
cracked = False
start = time.time()

for i in file.readlines():
    counter += 1
    guess = hash_sha256(i.strip())
    if guess == hashed_password:
        cracked = True
        print("Password cracked! The password is:", i.strip())
        print("It took", counter, "attempts to crack the password.")
        end = time.time()
        print("Time taken to crack the password:", round(end - start, 2), "seconds.")
        break

if not cracked:
    print("Password not found in wordlist")

Password cracked! The password is: ilovemycat
It took 78365 attempts to crack the password.
Time taken to crack the password: 0.38 seconds.


The cell below simulated the same brute-force attack, instead, where the password is hashed with a salt. As you can see, cracking a salted password takes almost 200 times longer than cracking a password without a salt. This is just with 1 byte salt. Increasing the salt by another byte would increase the time exponentially.

In [ ]:
import os, hashlib, time

def hash_sha256_salt(password: str, salt: bytes) -> str:
    sha256_hash = hashlib.sha256()
    sha256_hash.update(salt + password.encode())
    return sha256_hash.hexdigest(), salt.hex()

secret_password = input("Enter a password to crack: ")
salt = os.urandom(1) # two byte salt is more than enough

hashed_password = hash_sha256_salt(secret_password, salt)

salt_combinations = []

# for a in range(256):
#     value = bytes([a])
#     salt_combinations.append(value)

for a in range(256):
    for b in range(256):
        value = bytes([a, b])
        salt_combinations.append(value)

print("Total salt combinations to try:", len(salt_combinations))

file = open("100k_common_passwords.txt", "r")

common_passwords = [line.strip() for line in file.readlines()]

counter = 0
cracked = False
start = time.time()
for j in salt_combinations:
    if cracked:
        break
    for i in common_passwords:
        counter += 1

        guess = hash_sha256_salt(i.strip(), j)
        if guess == hashed_password:
            cracked = True
            print("Password cracked! The password is:", i.strip())
            print("It took", counter, "attempts to crack the password.")
            end = time.time()
            print("Time taken to crack the password:", round(end - start, 2), "seconds.")
            break

if not cracked:
    print("Password not found in wordlist")
    print("Tried", counter, "combinations.")
    print("Time taken to attempt all combinations:", round(time.time() - start, 2), "seconds.")

Total salt combinations to try: 65536
